In [1]:
BASE_PATH = ".."
RAW_PATH = f"{BASE_PATH}/data/raw"
CLEAN_PATH = f"{BASE_PATH}/data/clean"

In [2]:
import pandas as pd
import numpy as np

orders = pd.read_csv(f"{RAW_PATH}/olist_orders_dataset.csv")

In [3]:
orders.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [4]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [5]:
orders.describe(include='all')

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,98875,90733,81018,95664,459
top,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2018-08-02 12:05:26,2018-02-27 04:31:10,2018-05-09 15:48:00,2018-05-08 19:36:48,2017-12-20 00:00:00
freq,1,1,96478,3,9,47,3,522


In [6]:
orders.isnull().sum()


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Observed missing timestamps in approval and delivery stages, likely representing cancelled or in-progress orders. These records were retained to preserve funnel and logistics analysis integrity.

In [7]:
orders.duplicated().sum()

np.int64(0)

In [8]:
orders["order_id"].duplicated().sum()


np.int64(0)

No duplicate order records were found at both row-level and primary key level.

As observed all attributes are of object datatype. Now we'll Validate and enforce appropriate data types across identifiers, categorical variables, and timestamps to ensure accuracy in downstream temporal and aggregation analysis.

In [9]:
# IDs
orders["order_id"] = orders["order_id"].astype("string")
orders["customer_id"] = orders["customer_id"].astype("string")

# Status as category
orders["order_status"] = orders["order_status"].astype("category")

# Date columns
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")

orders.dtypes


order_id                         string[python]
customer_id                      string[python]
order_status                           category
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [10]:
orders["order_status"].value_counts()


order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

## Ensuring Timestamp Validity

In [11]:
invalid_dates = orders[
    (orders["order_delivered_customer_date"] < orders["order_purchase_timestamp"])
]

invalid_dates.shape


(0, 8)

In [12]:
orders["is_delivered"] = orders["order_status"] == "delivered"


In [13]:
orders.shape

(99441, 9)

In [14]:
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
is_delivered                        0
dtype: int64

In [15]:
orders.dtypes

order_id                         string[python]
customer_id                      string[python]
order_status                           category
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
is_delivered                               bool
dtype: object

In [17]:
import os

os.makedirs("../data/processed", exist_ok=True)

orders.to_csv(
    "../data/processed/orders_cleaned.csv",
    index=False
)


## Customers Data

In [3]:
customers = pd.read_csv(f"{RAW_PATH}/olist_customers_dataset.csv")

In [4]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [6]:
customers.isnull().sum()


customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

No Missing values

In [7]:
customers.duplicated().sum()


np.int64(0)

No duplicate data

## Validating Data Type

In [9]:
customers["customer_id"] = customers["customer_id"].astype("string")
customers["customer_unique_id"] = customers["customer_unique_id"].astype("string")
customers["customer_zip_code_prefix"] = customers["customer_zip_code_prefix"].astype("string")
customers["customer_city"] = customers["customer_city"].astype("string")
customers["customer_state"] = customers["customer_state"].astype("category")
customers.dtypes


customer_id                 string[python]
customer_unique_id          string[python]
customer_zip_code_prefix    string[python]
customer_city               string[python]
customer_state                    category
dtype: object

In [12]:
customers["customer_city"] = (
    customers["customer_city"]
    .str.strip()
    .str.lower()
)



In [13]:
customers["customer_state"].value_counts()


customer_state
SP    41746
RJ    12852
MG    11635
RS     5466
PR     5045
SC     3637
BA     3380
DF     2140
ES     2033
GO     2020
PE     1652
CE     1336
PA      975
MT      907
MA      747
MS      715
PB      536
PI      495
RN      485
AL      413
SE      350
TO      280
RO      253
AM      148
AC       81
AP       68
RR       46
Name: count, dtype: int64

Validated customer-level data by checking identifier uniqueness, standardizing location fields, and enforcing appropriate data types while preserving repeat-customer relationships

In [17]:
customers.duplicated().sum()
customers["customer_id"].duplicated().sum()


np.int64(0)

In [18]:
customers.shape

(99441, 5)

In [20]:
import os
os.makedirs("../data/processed", exist_ok=True)

customers.to_csv(
    "../data/processed/customers_cleaned.csv",
    index=False
)

Cleaned customer master data by validating identifier integrity, enforcing appropriate data types, and standardizing location fields. No missing or duplicate records were observed.

## Products Data

In [21]:
products = pd.read_csv(f"{RAW_PATH}/olist_products_dataset.csv")

In [22]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [23]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [24]:
products.isnull().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [25]:
products["product_category_name"] = (
    products["product_category_name"]
    .fillna("unknown")
    .str.strip()
    .str.lower()
)


In [28]:
products.columns = (
    products.columns
    .str.strip()
    .str.lower()
)


In [38]:
numeric_cols = [
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty",
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm"
]

for col in numeric_cols:
    products[col] = pd.to_numeric(products[col], errors="coerce")

In [39]:
products[numeric_cols].isnull().sum()


product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

Observed minimal missing values in product physical dimensions. These records were retained without imputation to avoid introducing bias, and will be conditionally filtered during logistics-specific analysis.

In [40]:
products.duplicated().sum()
products["product_id"].duplicated().sum()

np.int64(0)

In [41]:
products["product_id"] = products["product_id"].astype("string")
products["product_category_name"] = products["product_category_name"].astype("category")
products.dtypes

product_id                    string[python]
product_category_name               category
product_name_lenght                  float64
product_description_lenght           float64
product_photos_qty                   float64
product_weight_g                     float64
product_length_cm                    float64
product_height_cm                    float64
product_width_cm                     float64
dtype: object

In [42]:
import os
os.makedirs("../data/processed", exist_ok=True)

products.to_csv(
    "../data/processed/products_cleaned.csv",
    index=False
)


Cleaned product-level data by standardizing category labels, validating numeric attributes, and enforcing appropriate data types while retaining incomplete records for analysis-specific filtering.

## Payments Data

In [3]:
payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")

In [4]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [5]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


Column Name Standardization, in case there are any whitespaces.

In [6]:
payments.columns = (
    payments.columns
    .str.strip()
    .str.lower()
)


In [7]:
payments.isnull().sum()


order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [10]:
payments.duplicated().sum()
payments["order_id"].duplicated().sum()

np.int64(4446)

In [11]:
payments["order_id"] = payments["order_id"].astype("string")
payments["payment_type"] = payments["payment_type"].astype("category")

payments["payment_sequential"] = pd.to_numeric(
    payments["payment_sequential"], errors="coerce"
)

payments["payment_installments"] = pd.to_numeric(
    payments["payment_installments"], errors="coerce"
)

payments["payment_value"] = pd.to_numeric(
    payments["payment_value"], errors="coerce"
)
payments.dtypes

order_id                string[python]
payment_sequential               int64
payment_type                  category
payment_installments             int64
payment_value                  float64
dtype: object

In [12]:
payments["payment_type"].value_counts()


payment_type
credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: count, dtype: int64

In [13]:
payments[payments["payment_value"] <= 0]


,order_id,payment_sequential,payment_type,payment_installments,payment_value
19922,8bcbe01d44d147f901cd3192671144db,4,voucher,1,0.0
36822,fa65dad1b0e818e3ccc5cb0e39231352,14,voucher,1,0.0
43744,6ccb433e00daae1283ccc956189c82ae,4,voucher,1,0.0
51280,4637ca194b6387e2d538dc89b124b0ee,1,not_defined,1,0.0
57411,00b1cb0320190ca0daa2c88b35206009,1,not_defined,1,0.0
62674,45ed6e85398a87c253db47c2d9f48216,3,voucher,1,0.0
77885,fa65dad1b0e818e3ccc5cb0e39231352,13,voucher,1,0.0
94427,c8c528189310eaa44a745b8d9d26908b,1,not_defined,1,0.0
100766,b23878b3e8eb4d25a158f57d96331b18,4,voucher,1,0.0


Validated payment-level data by enforcing numeric and categorical data types, confirming absence of missing values, and preserving multi-record payment structures for installment and split-payment analysis.

In [14]:
payments["is_zero_payment"] = payments["payment_value"] == 0


In [18]:
payments["payment_type"].cat.categories
payments["payment_type"] = payments["payment_type"].cat.add_categories(["unknown"])


Handled undefined payment methods by explicitly extending categorical levels and assigning an ‘unknown’ category to preserve transactional completeness

In [21]:
payments["payment_type"] = (
    payments["payment_type"]
    .replace(["not_defined", "undefined", ""], "unknown")
)


C:\Users\Hp\AppData\Local\Temp\ipykernel_13936\376954407.py:3: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  .replace(["not_defined", "undefined", ""], "unknown")


In [24]:
payments["payment_type"].value_counts(dropna=False)


payment_type
credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
unknown            3
Name: count, dtype: int64

In [25]:
payments["payment_value"].describe()

count    103886.000000
mean        154.100380
std         217.494064
min           0.000000
25%          56.790000
50%         100.000000
75%         171.837500
max       13664.080000
Name: payment_value, dtype: float64

In [26]:
payments["is_zero_payment"].value_counts()

is_zero_payment
False    103877
True          9
Name: count, dtype: int64

In [27]:
payments.dtypes

order_id                string[python]
payment_sequential               int64
payment_type                  category
payment_installments             int64
payment_value                  float64
is_zero_payment                   bool
dtype: object

In [28]:
import os
os.makedirs("../data/processed", exist_ok=True)

payments.to_csv(
    "../data/processed/payments_cleaned.csv",
    index=False
)


In [29]:
order_items = pd.read_csv(f"{RAW_PATH}/olist_order_items_dataset.csv")

In [30]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [31]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [33]:
order_items.duplicated().sum()
order_items["order_id"].duplicated().sum()

np.int64(13984)

In [34]:
order_items.isnull().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [35]:
order_items.columns = (
    order_items.columns
    .str.strip()
    .str.lower()
)

In [37]:
order_items["order_id"] = order_items["order_id"].astype("string")
order_items["product_id"] = order_items["product_id"].astype("string")
order_items["seller_id"] = order_items["seller_id"].astype("string")

order_items["order_item_id"] = pd.to_numeric(
    order_items["order_item_id"], errors="coerce"
)

order_items["price"] = pd.to_numeric(
    order_items["price"], errors="coerce"
)

order_items["freight_value"] = pd.to_numeric(
    order_items["freight_value"], errors="coerce"
)

order_items["shipping_limit_date"] = pd.to_datetime(
    order_items["shipping_limit_date"], errors="coerce"
)
order_items.dtypes

order_id               string[python]
order_item_id                   int64
product_id             string[python]
seller_id              string[python]
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

In [38]:
order_items[order_items["price"] <= 0]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value


In [39]:
order_items[order_items["freight_value"] < 0]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value


In [40]:
order_items["item_total"] = order_items["price"] + order_items["freight_value"]

Validated order-item level data by enforcing numeric pricing fields, confirming absence of invalid monetary values, and preserving multi-item order structures for revenue analysis.

In [41]:
invalid_ship_dates = order_items[
    order_items["shipping_limit_date"].isna()
]

invalid_ship_dates.shape

(0, 8)

In [42]:
order_items.to_csv(
    "../data/processed/order_items_cleaned.csv",
    index=False
)

In [43]:
sellers = pd.read_csv(f"{RAW_PATH}/olist_sellers_dataset.csv")

In [44]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [45]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [49]:
sellers.columns = (
    sellers.columns
    .str.strip()
    .str.lower()
)


In [46]:
sellers.isnull().sum()

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

In [48]:
sellers.duplicated().sum()
sellers["seller_id"].duplicated().sum()

np.int64(0)

In [51]:
sellers["seller_id"] = sellers["seller_id"].astype("string")
sellers["seller_zip_code_prefix"] = sellers["seller_zip_code_prefix"].astype("string")
sellers["seller_city"] = sellers["seller_city"].astype("string")
sellers["seller_state"] = sellers["seller_state"].astype("category")
sellers.dtypes

seller_id                 string[python]
seller_zip_code_prefix    string[python]
seller_city               string[python]
seller_state                    category
dtype: object

In [52]:
sellers["seller_city"] = (
    sellers["seller_city"]
    .str.strip()
    .str.lower()
)
sellers["seller_state"].value_counts()

seller_state
SP    1849
PR     349
MG     244
SC     190
RJ     171
RS     129
GO      40
DF      30
ES      23
BA      19
CE      13
PE       9
PB       6
MS       5
RN       5
MT       4
RO       2
SE       2
AM       1
AC       1
PI       1
MA       1
PA       1
Name: count, dtype: int64

Cleaned seller master data by validating identifier uniqueness, enforcing appropriate data types, and standardizing geographic attributes to support location-based analysis.

In [53]:
sellers.to_csv(
    "../data/processed/sellers_cleaned.csv",
    index=False
)

In [3]:
reviews = pd.read_csv(f"{RAW_PATH}/olist_order_reviews_dataset.csv")

In [4]:
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [5]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [6]:
reviews.columns = (
    reviews.columns
    .str.strip()
    .str.lower()
)

In [7]:
reviews.isnull().sum().sort_values(ascending=False)


review_comment_title       87656
review_comment_message     58247
review_id                      0
review_score                   0
order_id                       0
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [8]:
reviews["review_id"] = reviews["review_id"].astype("string")

In [9]:
reviews.duplicated().sum()

np.int64(0)

In [10]:
reviews["review_id"].duplicated().sum()

np.int64(814)

In [11]:
reviews = reviews.drop_duplicates(subset=["review_id"])

In [12]:
reviews["order_id"].duplicated().sum()

np.int64(243)

In [13]:
reviews.groupby("order_id")["review_id"].nunique().sort_values(ascending=False).head()

order_id
3ab974ac744bd19a111d5e19f0a90649    2
60058a96e9d74aba1c66b86edbd0535e    2
142b0e3de54c110a3109b26c79d6ec2e    2
4d006fbdecd41058974d5d99f2d69650    2
78cd965d0bc0388d390404eee6490c5b    2
Name: review_id, dtype: int64

Observed duplicate review identifiers indicating technical record repetition, which were safely removed. Multiple reviews per order were retained as they represent legitimate customer feedback updates.

In [14]:
reviews["review_id"] = reviews["review_id"].astype("string")
reviews["order_id"] = reviews["order_id"].astype("string")

reviews["review_score"] = pd.to_numeric(
    reviews["review_score"], errors="coerce"
)

date_cols = [
    "review_creation_date",
    "review_answer_timestamp"
]

for col in date_cols:
    reviews[col] = pd.to_datetime(reviews[col], errors="coerce")
reviews.dtypes

review_id                  string[python]
order_id                   string[python]
review_score                        int64
review_comment_title               object
review_comment_message             object
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
dtype: object

In [15]:
reviews["review_score"].value_counts().sort_index()

review_score
1    11282
2     3114
3     8097
4    19007
5    56910
Name: count, dtype: int64

In [16]:
text_cols = ["review_comment_title", "review_comment_message"]

for col in text_cols:
    reviews[col] = (
        reviews[col]
        .astype("string")
        .str.strip()
    )

Prepared review-level data by validating rating integrity, preserving incomplete textual feedback, and enforcing timestamp consistency to support sentiment and satisfaction analysis.

In [17]:
reviews["review_id"].duplicated().sum()

np.int64(0)

In [18]:
import os
os.makedirs("../data/processed", exist_ok=True)

reviews.to_csv(
    "../data/processed/reviews_cleaned.csv",
    index=False
)
